In [ ]:
import keras
from keras.layers.merge import concatenate
from keras.layers import Input, Dense,Embedding,Flatten,Dropout,Dense
from keras.models import Model
from keras.optimizers import Adam
from keras.regularizers import l2
import pandas as pd
import numpy as np

Data available from http://files.grouplens.org/datasets/movielens/ml-latest-small.zip

In [ ]:
path = 'ml-latest-small/'

In [ ]:
ratings = pd.read_csv(path+'ratings.csv')
ratings.head()

In [ ]:
len(ratings)

In [ ]:
pd.read_csv(path+'movies.csv').head()

In [ ]:
movie_names = pd.read_csv(path+'movies.csv').set_index('movieId')['title'].to_dict()

In [ ]:
users = ratings.userId.unique()
movies = ratings.movieId.unique()

In [ ]:
userid2idx = {o:i for i,o in enumerate(users)}
movieid2idx = {o:i for i,o in enumerate(movies)}

In [ ]:
ratings.movieId = ratings.movieId.apply(lambda x: movieid2idx[x])
ratings.userId = ratings.userId.apply(lambda x: userid2idx[x])

In [ ]:
min_rating,max_rating = ratings.rating.min(),ratings.rating.max()
min_rating,max_rating

In [ ]:
n_users = ratings.userId.nunique()
n_movies = ratings.movieId.nunique()
n_users, n_movies

In [ ]:
n_factors = 50

In [ ]:
msk = np.random.rand(len(ratings)) < 0.8
trn = ratings[msk]
val = ratings[~msk]

In [ ]:
def embedding_input(name, n_in, n_out, reg):
    inp = Input(shape=(1,), dtype='int64', name=name)
    return inp, Embedding(n_in, n_out, input_length=1, embeddings_regularizer=l2(reg))(inp)

In [ ]:
user_in, u = embedding_input('user_in', n_users, n_factors, 1e-4)
movie_in, m = embedding_input('movie_in', n_movies, n_factors, 1e-4)

In [ ]:
x = concatenate([u, m])
x = Flatten()(x)
x = Dropout(0.3)(x)
x = Dense(70, activation='relu')(x)
x = Dropout(0.75)(x)
x = Dense(1)(x)
nn = Model([user_in, movie_in], x)
nn.compile(Adam(0.001), loss='mse')

In [ ]:
nn.fit([trn.userId, trn.movieId], trn.rating, batch_size=64, epochs=8, 
          validation_data=([val.userId, val.movieId], val.rating))